In [1]:
#data sources, etc
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import time

In [24]:
cData=pd.read_csv('127compiledData127.csv').drop(columns='Unnamed: 0')
cData

,date,open,high,low,close,volume,ticker
0,2020-06-08 16:00:00,9.5801,9.615,9.51,9.510,493476.0,SRTY
1,2020-06-08 15:45:00,9.6000,9.630,9.56,9.590,286957.0,SRTY
2,2020-06-08 15:30:00,9.6200,9.650,9.59,9.605,114534.0,SRTY
3,2020-06-08 15:15:00,9.5700,9.660,9.57,9.630,83371.0,SRTY
4,2020-06-08 15:00:00,9.5900,9.610,9.55,9.570,41627.0,SRTY
...,...,...,...,...,...,...,...
12673,2020-06-03 11:45:00,40.8500,41.060,40.84,40.920,440109.0,KRE
12674,2020-06-03 11:30:00,40.7800,40.900,40.69,40.850,251793.0,KRE
12675,2020-06-03 11:15:00,40.5700,40.840,40.53,40.780,591350.0,KRE
12676,2020-06-03 11:00:00,40.6400,40.660,40.48,40.570,252190.0,KRE


In [25]:
cData['ticker'].unique()

array(['SRTY', 'CAL', 'TEN', 'OSUR', 'APY', 'SABR', 'OVV', 'CIM', 'RUTH',
       'LILAK', 'UAA', 'MAC', 'TGI', 'SVC', 'MT', 'EB', 'AA', 'GPS',
       'XHR', 'PCG', 'AMKR', 'AZUL', 'ROIC', 'HBI', 'RLJ', 'PK', 'BPYU',
       'DENN', 'JBLU', 'BLMN', 'UGAZ', 'HST', 'KEY', 'FAZ', 'DLPH', 'SRG',
       'CXP', 'RRR', 'YEXT', 'PEB', 'AKR', 'PVAC', 'HOMB', 'ELY', 'DXD',
       'CUK', 'EPRT', 'CNP', 'PPC', 'XRX', 'AAL', 'GAN', 'NMIH', 'PLAY',
       'DFEN', 'SEAS', 'HA', 'OUT', 'VVV', 'TZA', 'JETS', 'OXY', 'ERX',
       'CCL', 'MGM', 'SDOW', 'DBX', 'WRI', 'NCLH', 'CCC', 'JWN', 'SAVE',
       'KTB', 'FSCT', 'USFD', 'AU', 'ILF', 'HGV', 'CHK', 'TRIP', 'PD',
       'CIT', 'HSBC', 'KSS', 'REM', 'YELP', 'HTA', 'CAKE', 'EWG', 'KDP',
       'PEAK', 'GM', 'SCHC', 'CAR', 'APLS', 'WFC', 'NAIL', 'WORK', 'KHC',
       'DUST', 'SPR', 'VTIQ', 'DAL', 'ZTO', 'RWM', 'TMF', 'CUZ', 'COLD',
       'TCBI', 'NKLA', 'KBE', 'WYND', 'DT', 'LUV', 'ADPT', 'URTY', 'ARES',
       'FR', 'WERN', 'UAL', 'ALK', 'SCHW', 'EFV', 'R

In [67]:
def ratioForwardMA (df,ma=5,maCol='open', forward=5):
    compiledDF=pd.DataFrame()
    for ticker in df['ticker'].unique():
        dfSub=df[df['ticker']==ticker]
        
        dfRatio = dfSub[['open','high', 'low','close']]/dfSub['open'].iloc[-1]
        dfRatio['upperShadow']=(dfRatio['high']-dfRatio[['open','close']].max(axis=1))
        dfRatio['lowerShadow']=(dfRatio['low']-dfRatio[['open','close']].min(axis=1))
        dfRatio['body']=dfRatio['close']-dfRatio['open']

        dfRatio['openF'+str(forward)]=dfRatio['open'].shift(forward)
        dfRatio['highF'+str(forward)]=dfRatio['high'].shift(forward)
        dfRatio['lowF'+str(forward)]=dfRatio['low'].shift(forward)
        dfRatio['closeF'+str(forward)]=dfRatio['close'].shift(forward)
        
        dfRatio['MA'+str(ma)]=dfRatio[maCol].rolling(window=ma).mean().shift(-ma+1)
        dfRatio.insert(0,'date',dfSub['date'])
        dfRatio.insert(1,'ticker', dfSub['ticker'])
        
        compiledDF=compiledDF.append(dfRatio)
        
    compiledDF=compiledDF.dropna()
    return compiledDF
        

In [72]:
rfmaDF=ratioForwardMA(cData)
rfmaDF

,date,ticker,open,high,low,close,upperShadow,lowerShadow,body,openF5,highF5,lowF5,closeF5,MA5
5,2020-06-08 14:45:00,SRTY,0.830736,0.835007,0.830736,0.832125,0.002882,0.000000,0.001389,0.831613,0.834643,0.825528,0.825528,0.829521
6,2020-06-08 14:30:00,SRTY,0.830736,0.834209,0.829000,0.830736,0.003472,-0.001736,0.000000,0.833341,0.835945,0.829868,0.832473,0.828132
7,2020-06-08 14:15:00,SRTY,0.831604,0.833341,0.829000,0.830728,0.001736,-0.001727,-0.000877,0.835077,0.837681,0.832473,0.833775,0.828132
8,2020-06-08 14:00:00,SRTY,0.827264,0.831466,0.826396,0.830736,0.000729,-0.000868,0.003472,0.830736,0.838549,0.830736,0.835945,0.827959
9,2020-06-08 13:45:00,SRTY,0.827264,0.829504,0.824660,0.827698,0.001806,-0.002604,0.000434,0.832473,0.834209,0.829000,0.830736,0.828479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12669,2020-06-03 12:45:00,KRE,1.012618,1.014968,1.009896,1.013360,0.001608,-0.002721,0.000742,1.005938,1.008164,1.005072,1.007917,1.012815
12670,2020-06-03 12:30:00,KRE,1.015586,1.015710,1.007917,1.012618,0.000124,-0.004701,-0.002969,1.009649,1.009896,1.004701,1.006185,1.012073
12671,2020-06-03 12:15:00,KRE,1.013112,1.016823,1.012370,1.015586,0.001237,-0.000742,0.002474,1.014349,1.015834,1.009401,1.009401,1.009698
12672,2020-06-03 12:00:00,KRE,1.012123,1.013731,1.008659,1.013360,0.000371,-0.003464,0.001237,1.014102,1.015339,1.012370,1.014349,1.008164


In [74]:
rfmaDF.to_csv('127CalculatedDataFrame.csv')